# MISP

In [1]:
%pip install pymisp


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
from dotenv import dotenv_values

In [4]:
from surianalytics.connectors import getGitRoot, check_str_bool

In [5]:
import os
import shutil

In [6]:
config = dotenv_values(os.path.join(getGitRoot(), ".env") if shutil.which("git") is not None else os.path.join(os.path.expanduser("~"), ".env"))

In [7]:
from pymisp import PyMISP

In [8]:
misp = PyMISP(config["MISP_HOST"],
              config["MISP_TOKEN"],
              check_str_bool(config["MISP_TLS_VERIFY"]),
              debug=False)

In [11]:
result = misp.search(controller='attributes', timestamp="1d", type_attribute="domain", category="Network activity", pythonify=False)

In [12]:
import pandas as pd

In [15]:
DF_ATTR = pd.json_normalize(result["Attribute"])

In [16]:
DF_ATTR.columns.values

array(['id', 'event_id', 'object_id', 'object_relation', 'category',
       'type', 'to_ids', 'uuid', 'timestamp', 'distribution',
       'sharing_group_id', 'comment', 'deleted', 'disable_correlation',
       'first_seen', 'last_seen', 'value', 'Tag', 'Event.org_id',
       'Event.distribution', 'Event.id', 'Event.info', 'Event.orgc_id',
       'Event.uuid'], dtype=object)

In [17]:
DF_ATTR.head(3)

,id,event_id,object_id,object_relation,category,type,to_ids,uuid,timestamp,distribution,...,first_seen,last_seen,value,Tag,Event.org_id,Event.distribution,Event.id,Event.info,Event.orgc_id,Event.uuid
0,328367,1646,0,None,Network activity,domain,True,58cc2154-dd22-4576-bc3e-d7d899eb82a3,1687242034,5,...,None,None,awd.byfaithchurch.org,"[{'id': '599', 'name': 'kill-chain:Command and...",2,1,1646,STI threat update - IoC edition,2,55887238-6ef4-49cb-93cc-f3575f84102b
1,328368,1646,0,None,Network activity,domain,True,2a5cab93-9bf7-4383-9407-5981ed580303,1687242034,5,...,None,None,thatanyusstnaptdatagrams.date,"[{'id': '599', 'name': 'kill-chain:Command and...",2,1,1646,STI threat update - IoC edition,2,55887238-6ef4-49cb-93cc-f3575f84102b
2,328369,1646,0,None,Network activity,domain,True,8f8f2bcb-89f0-4d5d-a63e-fa58b29097e8,1687242034,5,...,None,None,strickt.bit,"[{'id': '599', 'name': 'kill-chain:Command and...",2,1,1646,STI threat update - IoC edition,2,55887238-6ef4-49cb-93cc-f3575f84102b


In [18]:
DF_ATTR.groupby("type").agg({"value": "nunique",
                             "event_id": "nunique"})

,value,event_id
type,,
domain,5906,8


In [25]:
DF_ATTR.groupby(["Event.info", "event_id", "type"]).agg({"value": ["unique", "nunique"]})

value  \
                                                                                            unique   
Event.info                      event_id type                                                        
STI threat update - IoC edition 1646     domain  [awd.byfaithchurch.org, thatanyusstnaptdatagra...   
                                1647     domain  [awd.byfaithchurch.org, thatanyusstnaptdatagra...   
                                1648     domain  [awd.byfaithchurch.org, thatanyusstnaptdatagra...   
                                1649     domain  [awd.byfaithchurch.org, thatanyusstnaptdatagra...   
                                1650     domain  [awd.byfaithchurch.org, thatanyusstnaptdatagra...   
                                1651     domain  [awd.byfaithchurch.org, thatanyusstnaptdatagra...   
                                1652     domain  [awd.byfaithchurch.org, thatanyusstnaptdatagra...   
                                1653     domain  [awd.byfaithchurch.org, thatanyusstnaptdatagra...   

                                                         
                                                nunique  
Event.info                      event_id type            
STI threat update - IoC edition 1646     domain     998  
                                1647     domain     798  
                                1648     domain     399  
                                1649     domain    5906  
                                1650     domain    5906  
                                1651     domain    5906  
                                1652     domain    5906  
                                1653     domain    5906

# Scan for IoC values via scirius

In [26]:
from surianalytics.connectors import RESTSciriusConnector

In [27]:
c = RESTSciriusConnector()

Set timeframe to past 30 days.

In [45]:
c.set_query_delta(hours=24 * 1)

In [46]:
def gen_kql_or_domains(items: pd.Series) -> str:
    return " OR ".join(list(items.unique()))

In [56]:
bulksize = 100

In [73]:
L_IOC = list(DF_ATTR.value.unique())
len(L_IOC)

5906

In [74]:
DF_SNI = pd.DataFrame()

In [75]:
while True:
    batch = L_IOC[:bulksize]
    q_sni = "event_type: tls AND tls.sni.keyword: ({domains})".format(domains=" OR ".join(batch))
    df = c.get_events_df(qfilter=q_sni)
    print(len(df), len(batch))
    
    L_IOC = L_IOC[bulksize:]
    if len(L_IOC) < bulksize:
        print("done")
        break

0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
1000 100
1000 100
0 100
0 100
0 100
586 100
0 100
0 100
282 100
0 100
0 100
0 100
319 100
0 100
739 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
1000 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
455 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
0 100
